#Apache Spark Fundamentals: SQL/DataFrames

Spark SQL works with DataFrames. A DataFrame is a relational representation of data. It provides functions similar to SQL functions. Also, it allows us to write SQL-type queries.

DataFrame are similar to relation tables or DataFrames in python/R, but they have many optimizations that are executed "hidden" from the user. There are several ways to create DataFrames from collections, HIVE tables, relational tables, and RDDs.

In [0]:
from pyspark.sql.functions import col, countDistinct, sum, min, max, avg, desc

In [0]:
raw_fire_df = spark.read \
                .format("csv") \
                .option("header", "true") \
                .option("inferSchema", "true") \
                .load("/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv")

In [0]:
# Get the columns of the DataFrame
columns = raw_fire_df.columns
print(columns)

['Call Number', 'Unit ID', 'Incident Number', 'CallType', 'Call Date', 'Watch Date', 'Call Final Disposition', 'Available DtTm', 'Address', 'City', 'Zipcode of Incident', 'Battalion', 'Station Area', 'Box', 'OrigPriority', 'Priority', 'Final Priority', 'ALS Unit', 'Call Type Group', 'NumAlarms', 'UnitType', 'Unit sequence in call dispatch', 'Fire Prevention District', 'Supervisor District', 'Neighborhood', 'Location', 'RowID', 'Delay']


In [0]:
# We can modify the Columns using a common preprocessing
def preprocess_column_names(column):
    new_column = column.replace(" ", "")
    return new_column

new_columns = [preprocess_column_names(x) for x in columns]
print(new_columns)

['CallNumber', 'UnitID', 'IncidentNumber', 'CallType', 'CallDate', 'WatchDate', 'CallFinalDisposition', 'AvailableDtTm', 'Address', 'City', 'ZipcodeofIncident', 'Battalion', 'StationArea', 'Box', 'OrigPriority', 'Priority', 'FinalPriority', 'ALSUnit', 'CallTypeGroup', 'NumAlarms', 'UnitType', 'Unitsequenceincalldispatch', 'FirePreventionDistrict', 'SupervisorDistrict', 'Neighborhood', 'Location', 'RowID', 'Delay']


In [0]:
# Assign the new columns to the DataFrame
mapping = dict(zip(columns, new_columns))
fire_df = raw_fire_df.select([col(c).alias(mapping.get(c, c)) for c in columns])
print(fire_df.columns)

['CallNumber', 'UnitID', 'IncidentNumber', 'CallType', 'CallDate', 'WatchDate', 'CallFinalDisposition', 'AvailableDtTm', 'Address', 'City', 'ZipcodeofIncident', 'Battalion', 'StationArea', 'Box', 'OrigPriority', 'Priority', 'FinalPriority', 'ALSUnit', 'CallTypeGroup', 'NumAlarms', 'UnitType', 'Unitsequenceincalldispatch', 'FirePreventionDistrict', 'SupervisorDistrict', 'Neighborhood', 'Location', 'RowID', 'Delay']


In [0]:
fire_df.dtypes

Out[19]: [('CallNumber', 'int'),
 ('UnitID', 'string'),
 ('IncidentNumber', 'int'),
 ('CallType', 'string'),
 ('CallDate', 'date'),
 ('WatchDate', 'date'),
 ('CallFinalDisposition', 'string'),
 ('AvailableDtTm', 'string'),
 ('Address', 'string'),
 ('City', 'string'),
 ('ZipcodeofIncident', 'int'),
 ('Battalion', 'string'),
 ('StationArea', 'string'),
 ('Box', 'string'),
 ('OrigPriority', 'string'),
 ('Priority', 'string'),
 ('FinalPriority', 'int'),
 ('ALSUnit', 'boolean'),
 ('CallTypeGroup', 'string'),
 ('NumAlarms', 'int'),
 ('UnitType', 'string'),
 ('Unitsequenceincalldispatch', 'int'),
 ('FirePreventionDistrict', 'string'),
 ('SupervisorDistrict', 'string'),
 ('Neighborhood', 'string'),
 ('Location', 'string'),
 ('RowID', 'string'),
 ('Delay', 'double')]

In [0]:
# where is an alias for filter

In [0]:
# We can drop a particular column in a DataFrame
new_fire_df = fire_df.drop("WatchDate")
print(new_fire_df.columns)

['CallNumber', 'UnitID', 'IncidentNumber', 'CallType', 'CallDate', 'CallFinalDisposition', 'AvailableDtTm', 'Address', 'City', 'ZipcodeofIncident', 'Battalion', 'StationArea', 'Box', 'OrigPriority', 'Priority', 'FinalPriority', 'ALSUnit', 'CallTypeGroup', 'NumAlarms', 'UnitType', 'Unitsequenceincalldispatch', 'FirePreventionDistrict', 'SupervisorDistrict', 'Neighborhood', 'Location', 'RowID', 'Delay']


In [0]:
# Aggregation functions and alias
agg_df = (
    new_fire_df
        .where("ZipcodeofIncident IS NOT NULL")
        .groupBy("ZipcodeofIncident")
        .agg(
            countDistinct("CallNumber").alias("num_incidents"),
            sum("Delay").alias("sum_delays"),
            max("Delay").alias("max_delays"),
            min("Delay").alias("min_delays"),
            avg("Delay").alias("avg_delays"),
        ).sort(desc(col("avg_delays")))
)
display(agg_df.limit(10))

ZipcodeofIncident,num_incidents,sum_delays,max_delays,min_delays,avg_delays
94158,9273,136712.0333333335,1503.7666666666667,0.016666666666666666,6.520343078806386
94130,10552,150000.56666666665,1016.15,0.03333333333333333,5.497345403014976
94129,4663,60397.600000000006,123.98333333333333,0.016666666666666666,4.9510287728502345
94131,36826,342028.2166666665,1497.8,0.016666666666666666,4.1626489870100345
94124,110049,956264.15,1739.4333333333334,0.016666666666666666,4.161940730488675
94132,49815,446596.94999999984,628.55,0.016666666666666666,4.119746042581453
94134,58918,500654.63333333284,1126.5,0.016666666666666666,4.084042755680269
94127,21731,189892.5833333333,958.8166666666667,0.016666666666666666,4.068574622015583
94121,52793,472129.3000000003,508.85,0.016666666666666666,4.037363605267661
94105,49427,420187.3333333328,683.2666666666667,0.06666666666666667,4.018662509524123


In [0]:
# JOIN types
"""
df1 = df1.join(df2, df1["id"] == df2["id"], "inner")
"""

Out[31]: '\ndf1 = df1.join(df2, df1["id"] == df2["id"], "inner")\n'